In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import transformers
from datasets import load_dataset, ClassLabel
from transformers import AutoModelForSequenceClassification, AutoTokenizer, pipeline

In [ ]:
train_dataset = load_dataset("winvoker/turkish-sentiment-analysis-dataset", split="test[:1%]")
test_dataset = load_dataset("winvoker/turkish-sentiment-analysis-dataset", split="test[1:2%]")

Using custom data configuration winvoker--turkish-sentiment-analysis-dataset-dbdc52d492aad425
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/winvoker--turkish-sentiment-analysis-dataset-dbdc52d492aad425/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)
Using custom data configuration winvoker--turkish-sentiment-analysis-dataset-dbdc52d492aad425
Reusing dataset csv (/root/.cache/huggingface/datasets/csv/winvoker--turkish-sentiment-analysis-dataset-dbdc52d492aad425/0.0.0/433e0ccc46f9880962cc2b12065189766fbb2bee57a221866138fb9203c83519)


In [ ]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
labels = ClassLabel(names_file='labels.txt')

def preprocess_function(batch):
    tokens = tokenizer(batch["text"], truncation=True, padding="max_length", max_length=256)
    tokens["label"] = labels.str2int(batch["label"])
    return tokens

In [ ]:
import torch

def prepare_dataset(dataset):
  dataset = dataset.map(preprocess_function, batched=True)
  dataset.set_format(type='torch', columns=['input_ids', 'token_type_ids', 'attention_mask', 'label'])
  dataloader = torch.utils.data.DataLoader(dataset, batch_size=8)
  return dataloader
  
train_dataloader = prepare_dataset(train_dataset)
test_dataloader = prepare_dataset(test_dataset)

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [ ]:
import torch
from torch.optim import AdamW
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=3)
optimizer = AdamW(model.parameters(), lr=5e-5)

Downloading:   0%|          | 0.00/416M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [ ]:
from transformers import get_scheduler

num_epochs = 3
num_training_steps = num_epochs * len(train_dataloader)
lr_scheduler = get_scheduler(
    name="linear", optimizer=optimizer, num_warmup_steps=0, num_training_steps=num_training_steps
)

In [ ]:
import torch

device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
print("done")

done


In [ ]:
from tqdm.auto import tqdm

progress_bar = tqdm(range(num_training_steps))

model.train()
for epoch in range(num_epochs):
    for batch in train_dataloader:
        batch = {k: v.to(device) for k, v in batch.items()}

        labels = batch["label"]
        input_ids = batch["input_ids"]
        token_type_ids = batch["token_type_ids"]
        attention_mask = batch["attention_mask"]

        output = model(input_ids, 
                             token_type_ids=token_type_ids, 
                             attention_mask=attention_mask, 
                             labels=labels)
        loss = output.loss
        loss.backward()

        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

        progress_bar.update(1)
        
    # Compute Metrics for test set
    for batch in test_dataloader:

        batch = {k: v.to(device) for k, v in batch.items()}
        labels = batch["label"]
        input_ids = batch["input_ids"]
        token_type_ids = batch["token_type_ids"]
        attention_mask = batch["attention_mask"]

        with torch.no_grad():
            output = model(input_ids, 
                                token_type_ids=token_type_ids, 
                                attention_mask=attention_mask, 
                                labels=labels)

        logits = output.logits
        predictions = torch.argmax(logits, dim=-1)
        metric.add_batch(predictions=predictions, references=batch["label"])
    print("Test Accuracy : ", metric.compute()["accuracy"])

  0%|          | 0/186 [00:00<?, ?it/s]